In [2]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm
from scipy.signal import convolve

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# Obviously it is not a great idea to ignore warnings, however there are quite many
# RuntimeWarnings because of division by 0 in some parts of this notebook. To keep
# the instructive nature of this example notebook, these warnings are ignored.

import time
start_time = time.time()

In [6]:
# detection parameters
Npix_min = 20*6*5
Npix_max = 500*6*5
OW_thr_factor =-0.2

sigma = 9 # smoothing parameter

In [5]:
datestart, dateend = "2012-04-10", "2012-05-04"
#periods = [
#    ("2012-01-01", "2012-01-25"), ("2012-01-26", "2012-02-19"), ("2012-02-20", "2012-03-15"),
#    ("2012-03-16", "2012-04-09"), ("2012-04-10", "2012-05-04"), ("2012-05-05", "2012-05-29"),
#    ("2012-05-30", "2012-06-28"), ("2012-10-27", "2012-11-20"), ("2012-11-21", "2012-12-15"),
#    
#    ("2012-06-29", "2012-07-28"), ("2012-07-29", "2012-08-27"), ("2012-08-28", "2012-09-26"), ("2012-09-27", "2012-10-26"),
#]

In [4]:
depth= 0  #corresponding to... 
depth_index = 0 # tracking only on surface

#mesh_mask = xr.open_dataset('/gxfs_work/geomar/smomw355/model_data/ocean-only/INALT60.L120-KRS0020/nemo/suppl/2_INALT60.L120-KRS0020_mesh_mask.nc') 
#depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in range(0, 64, 3)] #upper 1000m, otherwise: range(0, len(mesh_mask.nav_lev.values), 3)
#print(depth_information)

In [4]:
experiment_name = 'INALT60.L120-KRS0020'
data_resolution = '1d'

# TRACKING

Things to note for the setting of `tracking_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the detected eddies. In our case here, for the year 0002 and a 5-day temporal resolution of the data, this is `'0002-01-05'` (The `MITgcm` stores the 5-daily averages at the end of the 5-day period).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. If you stored the detected eddies in files and want to track these, set `'dict': 0`, make sure `'data_path'`, `'file_root'` and `'file_spec'` are set accordingly and use `in_file=True` as an argument to `et.tracking.track()`.. The method will look for files `datapath + file_root + 'YYYYMMDD' + file_spec + '.pickle'`, the date `'YYYYMMDD'` is automatically calculated from `'start_time'`, `'dt'`, and `'end_time'`. You have to make sure that the stored, detected eddies contain that date in their filename (e.g. as defined in the cell above)!  

Some notes on the search distance `search_dist` that is used to determine in what radius to look for a similar eddy at the next time step.  
    - If `search_circle: True`, the algorithm simply searches for similar eddies within a radius of `search_dist` kilometers around the center of the current eddy. This is the simplest method.
    - If `search_circle: False`, the algorithm will determine where to look for similar eddies based on an ellipse with a minor axis of `search_dist` kilometers. If `search_dist: 0`, the minor and major axes of the ellipse are calculated based on the propagation of Rossby waves to account for the fact that eddies will move towards the West (see [Chelton et al., 2011](https://www.sciencedirect.com/science/article/pii/S0079661111000036) for details). In regions with strong currents this might lead to a loss of a lot of tracks, but no sensitivity studies have been conducted... 

In [15]:
datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'   ## !! SMOOTHED !!

In [16]:
# Specify parameters for eddy tracking
tracking_parameters = {'model': 'ORCA',
                       'grid': 'latlon',
                       'start_time': datestart, # time range start
                       'end_time': dateend, # time range end
                       'calendar': 'standard', # calendar, must be either 360_day or standard
                       'dt': 1, # temporal resolution of the data in days
                       'lon1': 0, # minimum longitude of detection region
                       'lon2': 40, # maximum longitude
                       'lat1': -45, # minimum latitude
                       'lat2': -25, # maximum latitude
                       'search_dist': 50., # maximum distance of search ellipse/circle from eddy center in km
                                          # if ellipse: towards the east (if set to 0, it
                                          # will be calculated as (150. / (7. / dt)))
                       # max vel 3.5 m/s --> 50km in 4 hours
                       'search_circle': True, # if True, search in a circle. otherwise use ellipse
                       'eddy_scale_min': 0.5, # minimum factor by which eddy amplitude and area are allowed to change in one timestep
                       'eddy_scale_max': 1.5, # maximum factor by which eddy amplitude and area are allowed to change in one timestep
                       'dict': 0, # eddies dictionary containing detected eddies to be used when not stored in files (set to 0 otherwise)
                       'data_path': datapath, # path to the detected eddies pickle files
                       'file_root': 'Eddies',
                       'file_spec': f'OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}',
                       'ross_path': '/gxfs_work/geomar/smomw523/eddytools/'} # path to rossrad.dat containing Chelton et a1. 1998 Rossby radii

# detected eddies are loaded from file with the filename
# 'trac_param['data_path'] + trac_param['file_root'] + '_'
# + str(datestring) + '_' + trac_param['file_spec'] + '.pickle'
# `datestring` is created from the `time` value of the eddy

In [17]:
# Now we track the eddies, all information needed has to be added to `tracking_parameters`
tracks = et.tracking.track(tracking_parameters)

no eddies found to track


We now have tracked all eddies that met the criteria specified in `tracking_parameters`. Every entry `i` in `tracks[i]` corresponds to one complete track.

In [9]:
# The entries in `track` look like this
tracks[1]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# To have a look at how the tracking performs, just pick two eddies and see whether they are tracked.
ed1 = 0
ed2 = 4
t = 0
j = 5

plt.figure(figsize=(18, j))

plot_lon = data_int['lon'].where(data_int['lon'].values > 0, other=data_int['lon'].values + 360)

ed1_lon = tracks[ed1]['lon']
ed1_lon[ed1_lon < 0] = ed1_lon[ed1_lon < 0] + 360
ed2_lon = tracks[ed2]['lon']
ed2_lon[ed2_lon < 0] = ed2_lon[ed2_lon < 0] + 360

for i in np.arange(0, j):
    plt.subplot(1, j, i + 1)
    plt.pcolormesh(plot_lon, data_int.lat, data_int.OW.isel(time=t + i).values,
                   vmin=-5e-10, vmax=5e-10, cmap=cm.balance, shading='auto')
    plt.plot(ed1_lon[t:t + i+1], tracks[ed1]['lat'][t:t + i+1], marker='o', color='m')
    plt.plot(ed2_lon[t:t + i+1], tracks[ed2]['lat'][t:t + i+1], marker='o', color='y')

In [ ]:
 # We save the tracks for later use
with open(datapath
          + 'test_19760101_19761231_tracks_OW0.3'
          + '_test.pickle', 'wb') as f:
    pickle.dump(tracks, f, pickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
# This is how to open the tracks-file again (no need to do that if we just saved it)
with open(datapath
          + 'test_19760101_19761231_tracks_OW0.3'
          + '_test.pickle', 'rb') as f:
    tracks = pickle.load(f)
f.close()

### Split up the tracking
If detecting and tracking in larger regions, the tracking can take very long. Because computations are often limited in duration on high-performance computing clusters, an option to split up the tracking into several chunks has been added. Note that the tracking still needs to be done serially, one chunk after the other.

We first track the first half of the month, using the function `split_track()`.

The file `tmp_split1` contains the tracks for the first time span as if they were tracked with `track()`. `tracks_split` additionally contains all unfinished tracks and needs to be passed on to the next round of tracking. `terminated1` contains all tracks that have been already terminated and needs to passed on to the next round as well to make sure they don't get tracked again.

The variable `tracks_from_split` now contains the same tracks as the variable `tracks` from the normal tracking function `track()`. `tracks_split2` and `terminated2` are only needed when another period of tracking is to be added.

### SAMPLING

Things to note for the setting of `sample_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the tracked eddies. In our case here, for the year 1976 and a 5-day temporal resolution of the data, this is `'1976-01-03'` (if we had daily data, it would be `'1976-01-01'`).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. Right now, the usage of `'range'` and `'split'` has not been thouroughly tested! It seems to work for most cases though.  

`'range'`: Set to `True` if you only want to sample eddies within a certain range `'values_range'` of a property `'var_range'` in the dataset `'ds_range'`. `'var_range'` needs to be 2D (thus the `.isel(z=9)` in the example below) and interpolated to the same grid as `OW` used above. It is most likely that, if you follow this example, `'var_range'` is stored in the same dataset as `OW`. In the example below, only eddies that have a center temperature between 4 and 7 degrees C at depth level 10 (`z=9`) will be sampled and stored.  

`'split'`: Set to `True` if you want to split the sampled eddies into two categories, above and below a certain threshold value `'value_split'` of a variable `'var_split'` in the dataset `'ds_split'`. As for `'range'`, `'var_split'` needs to be 2D and interpolated to the same grid as `OW` used above. In the example below the eddies will be put into two categories: In the first category, the eddies must have a center surface salinity above 34.0 and in the second category, below 34.0.

The sampling can take quite long as for every eddy that fits the criteria we need to read data from disk at every time step  
The names of the files will be defined by `save_name`, the criteria you specify and the eddy number  
For this sampling parameters, the file name of the first eddy will be  
`test.anticyclonic.larger_25.longer_5.0000001.nc`  

If you set `'split'` to `True`, the file names will differ for the two categories.  
`test.anticyclonic.larger_25.longer_5.0000001.above_thr.nc` for eddies that are above `'value_split'`  
`test.anticyclonic.larger_25.longer_5.0000001.below_thr.nc`
for eddies that are below `'value_split'`.

# AVERAGING

We can now average over the sampled eddies.  
As each eddy has its own file, we first need to find out how many files/samples there are, so we can loop over them and then store the datasets in a dictionary.  
Note that for large region, i.e. a lot of samples the resulting dataset could be too large to fit into memory. The `chunks={}` argument to `xr.open_dataset()` is an attempt to overcome this, however I do not know yet whether this has a lot of effect! One could also split the samples into several parts and then work on each part seperately (the number of eddies going into each average is stored, so one could later do a weighted average over the different parts!)

First a "preparation" is performed: Basically, all eddies are interpolated onto a normalized (in length) section crossing them through the eddy center. This section's orientation can be defined by the argument `section`, which can be either `'zonal'` (default) or `'meridional'`. The length of this section can be defined with argument `interp_vec`. A larger number gives a finer resolution of the interpolated data, but might not necessarily be useful (if input data is coarser resolution for example).    
For every variable specified, at every time step, the values and anomalies (with respect to surroundings) are interpolated (with method `method`) onto the normalized section and stored according to the month the eddy was first detected. The depth profile of the surroundings is stored as well.  
For available interpolation methods please have a look in the documentation of the underlying function [`scipy.interpolate.interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html).

Here for example, 5 eddies originating in July have been stored with a maximum length of 37 time steps. 11 is the length of the depth dimension, 41 the length of the normalized section.

Next we can average these samples in different ways.  
Three possibilities are given with `average`  
1. seasonal -> bins the samples (of all available years) into four seasons (DJF, MAM, JJA, SON) and averages them into a seasonal climatology  
2. monthly  -> does the same for each month (so you end up with a monthly climatology)  
3. total    -> averages over all available eddies  
The three methods all return the means, standard deviations (across eddies), and the number of eddies that went into the derived quantities for a) the variables specified, b) their anomalies to the surroundings, c) the surroundings. These results are stored in the output dictionary under `output['ave']['mean'][period]['variable']`, `output['ave']['mean'][period]['variable_anom']`, `output['ave']['mean'][period]['variable_around']`, respectively. `period` refers to 1. `DJF`, `MAM`, `JJA` or `SON`, 2. `01`, `02`, `03`, etc., representing the month, or in case of the total average thie layer in the dictionary does not exist.  
Additionally, the averaged temporal evolution of the eddy centers will be stored under `output['evo'][...]`.